In [2]:
import pandas as pd
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import time
plt.rc('font', size=12)

# Filter out deprecated warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('../kennedy/victoria.csv')
data1 = data

In [4]:
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

In [5]:
vic = data[[TIME, PRICE, GENERATION, DEMAND]]

# Since the first date is at 00:00:00, the first period should be 48
period = [48]
x = 1
while x < len(vic):
    for i in range(48):
        period.append(i+1)
        x += 1

vic.insert(1, 'Period', period)

In [6]:
def first_cycle(spot_price):
    """ Returns first indexes of the first periods for the first cycles """
    
    max_price = 0
    min_price = 999999

    for i in range(48-6):
        """ Finds which 6 periods have the most sum and least sum 
            by going through 1 to 6, 2 to 7 and so on """

        curr = spot_price.iloc[i:i+6 ,0].sum()

        if curr < min_price:
            min_price = curr
            min_i = i  # Stores the first index of the max period

        if curr > max_price:

            max_price = curr
            max_i = i  # Stores the first index of the min period
            
    min_, max_ = store_index(min_i, max_i)
                
    return min_, max_

In [7]:
def sec_cycle(spot_price, min_index, max_index):
    """ Returns first indexes of the first periods for the second cycles """
    
    remaining = list(spot_price.index)
    sec_min_price = 999999
    sec_max_price = 0
    to_remove = min_index + max_index
    
    # remove all periods after max
    remaining = remaining[:remaining.index(to_remove[-1])+1]
    
    for index in to_remove:
        remaining.remove(index)

    for i in range(len(remaining) - 6):

        # make sure the next six indexes are increment of 1
        if remaining[i] == (remaining[i+5] - 5):

            curr_sum = spot_price.iloc[remaining[i]:remaining[i]+6 ,0].sum()

            if curr_sum < sec_min_price:
                sec_min_price = curr_sum
                sec_min_i = remaining[i]  # Stores the first index of the second max period

            if curr_sum > sec_max_price:

                sec_max_price = curr_sum
                sec_max_i = remaining[i]  # Stores the first index of the second min period
                
    min_, max_ = store_index(sec_min_i, sec_max_i)
    
    min_index += min_
    max_index += max_
                
    return min_index, max_index

In [8]:
def store_index(index1, index2):
    """ Store the rest of the max and min price indexes """
    list1 = []
    list2 = []
    for i in range(6):
        list1.append(index1 + i)
        list2.append(index2 + i)
        
    return list1, list2

In [9]:
def algorithm2(ori_df):
    """ Finds optimal charge and discharge period from the mean """
    
    spot_price = ori_df.groupby(['Period'])[[PRICE]].mean()
    
    # First cycle
    min_index, max_index = first_cycle(spot_price)

    # Second cycle
    # Comment line 11 if only want one cycle
    min_index, max_index = sec_cycle(spot_price, min_index, max_index)

    # the charge and discharge period are fixed in Algorithm 2, +1 to get their periods
    charge_period = list(np.asarray(min_index) + 1)
    discharge_period = list(np.asarray(max_index) + 1)
    
    return charge_period, discharge_period, 

In [10]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, 'Period', PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['revenue'] = 0
    
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous day
    df = df.drop([0], axis=0) 
    
    return df

In [11]:
def find_all(ori_df):
    """ Returns a completed dataframe """
    """ This is the main function, calling this function will automatically run all other functions """
    
    start = time.time()
    
    charge_period, discharge_period = algorithm2(ori_df)
    
    df = create_df(ori_df)
    
    for i in list(df.index):

        period = df.at[i, "Period"]
        price = df.at[i, PRICE]

        """ Find Opening Cap """
        # Starts from 1 because the first row was removed
        if i != 1:
            df.at[i,"opening"] = df.at[i-1,"closing"]

        opening_cap = math.ceil(df.at[i, "opening"])




        """ Find raw_power """
        if period in charge_period:
            df.at[i, "raw_power"] = -math.floor(min(POWER,(CAPACITY-opening_cap)/(CHARGE_EFF/100)*2))

        elif period in discharge_period:
            df.at[i, "raw_power"] = math.floor(min(POWER,opening_cap/(DISCHARGE_EFF/100)*2))

        rawPower = df.at[i, "raw_power"]




        """ Find dispatch """
        if rawPower < 0:
            eff = 1

        else:
            eff = DISCHARGE_EFF / 100

        df.at[i,"dispatch"] = math.ceil(rawPower / 2 * eff)
        dispatch = df.at[i, "dispatch"]



        """ Find Closing Cap """
        if dispatch < 0:
            eff = CHARGE_EFF / 100

        else:
            eff = 100 / DISCHARGE_EFF

        df.at[i,"closing"] = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))



        """ Find revenue """
        if dispatch < 0:
            factor = 1/MLF

        else:
            factor = MLF

        df.at[i,"revenue"] = math.ceil(price * dispatch * factor)
        
    
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    end = time.time()
    print("Time Complexity for running the entire Algorithm 2: {time_taken}s".format(time_taken = end-start))
    
    return df

In [12]:
# Using mean
alg = find_all(vic)

Total revenue in the dataset: 62211575
Total days in the dataset: 1322.0
Revenue per day: 47058.68003025719
Time Complexity for running the entire Algorithm 2: 3.465169668197632s


In [13]:
# Using median
alg = find_all(vic)

Total revenue in the dataset: 62211575
Total days in the dataset: 1322.0
Revenue per day: 47058.68003025719
Time Complexity for running the entire Algorithm 2: 3.468128204345703s


In [14]:
deliverable3 = vic[(vic[TIME] >= "2020-07-17 00:00:00") & (vic[TIME] < "2020-07-18 00:00:30")]
deliverable3 = deliverable3.reset_index(drop=True)
del3 = find_all(deliverable3)

Total revenue in the dataset: 44471
Total days in the dataset: 1.0
Revenue per day: 44471.0
Time Complexity for running the entire Algorithm 2: 0.01605534553527832s


# Include other factors (seasons) into our algorithm